In [20]:
import pandas as pd
import csv

#"POD"
pod = pd.read_csv("open purchase orders.csv", encoding="utf-8", encoding_errors="replace")
pod.drop(columns=['Name', 'Amount', 'Open Balance', "Rcv'd", "Qty", "Memo"], inplace=True)
pod.rename(columns={"Date": "Order Date", "Num": "QB Num", "Source Name": "Name", "Backordered": "Qty(+)"}, inplace=True)
pod.drop(pod.columns[0], axis=1, inplace=True)
pod.dropna(how='all', inplace=True)
pod.dropna(thresh=5, inplace=True) #刪除有效值少於5個的行
# print(pod['Item'])
pod['Item'] = pod['Item'].str.split(':').str[1]
# print(pod['QB Num'])
pod['QB Num'] = pod['QB Num'].str.split('(').str[0]
for col in ['Order Date', 'Deliv Date']:
    pod[col] = pd.to_datetime(pod[col]).dt.strftime('%Y/%m/%d')
pod.to_csv('open purchase2.csv', index=False)


#"NAV"
NAV = pd.read_csv("Sales Date return platform.csv", usecols=['Document No.', "Customer PO No.", "Customer Ordering Model",
                                                             "OP Estimated Shipping Date", "Quantity", "No.",
                                                             "Customer Ordering Desc."], encoding='utf-8')
NAV.rename(columns={"Customer PO No.": "QB Num", "Customer Ordering Model": "Item", 'Document No.': "Remark",
                    "OP Estimated Shipping Date": "Ship Date", "Quantity": "Qty(+)"}, inplace=True)
NAV = NAV[NAV['Item'] != 'Engineer Service- COS']
NAV = NAV[NAV['Item'] != 'CUSTOMER SERVICES']
NAV = NAV[NAV['Item'] != 'FORWARDING CHARGE, EXCLUDING IMPORT DUTY.']
# print(NAV['QB Num'])
NAV['QB Num'] = NAV['QB Num'].str.split('(').str[0]

NAV.to_csv('NAV1.csv', index=False)

print(len(NAV))

223


In [8]:
# # 讀取 NAV1 並篩選符合條件的數據
# s50 = []

# for _, row in NAV.iterrows():
#     if row['No.'].startswith("S"):  # 檢查 Item 是否以 "S" 開頭
#         s50.append(row)

# result_lists = []
# for original_list in s50:
#     # 分割字串
#     product_str = original_list[-1]
#     product_str = product_str.replace('\u00A0', ' ').replace('\u3000', ' ')
#     product_info = product_str.split(', including ')
#     #product_info = original_list[-1].split(', including ')
#     # print(product_info)
#     # product_info[0] = product_info[0].split(',')[0]  # 產品代碼
#     components = product_info[1].split(', ') if len(product_info) > 1 else []
#     # print(components)
    
#     ## Append component to result_lists
#     # 建立各組件的新 list
#     for component in components:
#         new_list = original_list.copy()
#         new_list['Customer Ordering Desc.'] = component.strip()
#         result_lists.append(new_list)
   
#     ## Append base to result_lists
#     # 加入產品代碼(Change Desc. to product_info[0])
#     new_list_with_product_code = original_list.copy()
#     new_list_with_product_code['Customer Ordering Desc.'] = product_info[0]
#     result_lists.append(new_list_with_product_code)
    

# for i in range(0,len(result_lists)):
#     result_lists[i]['Item'] = result_lists[i]['Customer Ordering Desc.']


# # 調整數據格式
# transformed_lists = []
# for result_list in result_lists:
#     transformed_list = result_list.copy()
#     transformed_list['Item'] = transformed_list['Item'].replace(" ", "")
    
#     if len(transformed_list['Item']) > 1 and transformed_list['Item'][1] == 'x' and transformed_list['Item'][0].isdigit():
#         quantity = int(transformed_list['Item'].split('x')[0])
#         name = transformed_list['Item'].split('x')[-1]
#         transformed_list['Item'] = name
#         transformed_list['Qty(+)'] = str(quantity * float(transformed_list[4]))  # 更新數量

#     transformed_lists.append(transformed_list)

# print(pd.DataFrame(transformed_lists))


### ChatGPT 改良的版本如下

In [21]:
import re
import pandas as pd

# Keep only rows where No. starts with 'S'
df = NAV.loc[NAV['No.'].astype(str).str.startswith('S', na=False)].copy()
df = df.fillna('')

result = []

for rec in df.to_dict('records'):
    # Parse "Customer Ordering Desc."
    desc = (rec['Customer Ordering Desc.']
            .replace('\u00A0', ' ')
            .replace('\u3000', ' ')
            .strip())

    # base + components
    parts = re.split(r',\s*including\s*', desc, maxsplit=1, flags=re.I)
    base = parts[0].split(',', 1)[0].strip()
    comps = [c.strip() for c in parts[1].split(',')] if len(parts) > 1 else []

    # Build rows: components + base (dedup preserves order)
    targets = list(dict.fromkeys([*comps, base]))

    for t in targets:
        new = rec.copy()
        new['Customer Ordering Desc.'] = t
        new['Item'] = t  # mirror

        # Expand patterns like "2xSSD"
        compact = t.replace(' ', '')
        m = re.match(r'^(\d+)x(.+)$', compact)
        if m:
            mult = int(m.group(1))
            new['Item'] = m.group(2)
            try:
                new['Qty(+)'] = float(new['Qty(+)']) * mult
            except Exception:
                new['Qty(+)'] = mult

        result.append(new)

transformed_lists = pd.DataFrame(result, columns=['Remark','QB Num','No.','Item','Qty(+)','Ship Date','Customer Ordering Desc.'])
print(transformed_lists.to_string(index=False))
len(transformed_lists)


    Remark     QB Num              No.                                                 Item  Qty(+)  Ship Date                              Customer Ordering Desc.
SO25050035 POD-250648 S51-SL1708FF-003                                            i7-9700TE    10.0 2025/11/19                                            i7-9700TE
SO25050035 POD-250648 S51-SL1708FF-003                                      DDR4-16GB-32-SM    10.0 2025/11/19                                      DDR4-16GB-32-SM
SO25050035 POD-250648 S51-SL1708FF-003                         M.280-SSD-1TB-PCIe44-TLC5-PN    10.0 2025/11/19                         M.280-SSD-1TB-PCIe44-TLC5-PN
SO25050035 POD-250648 S51-SL1708FF-003                                        SEMIL-1708-FF    10.0 2025/11/19                                        SEMIL-1708-FF
SO25050036 POD-250649 S51-SL1708FF-003                                            i7-9700TE    10.0 2025/12/17                                            i7-9700TE
SO25050036 POD-2

184

In [10]:
import os, csv

file = 'NAV1.csv'
rows_to_add = transformed_lists  # your list of rows

with open(file, 'a+', encoding='utf-8', newline='') as f:
    f.seek(0)                                  # move to start to read
    before = sum(1 for _ in csv.reader(f))     # count existing rows (incl. header if any)

    # go back to end to append
    f.seek(0, os.SEEK_END)
    w = csv.writer(f)
    w.writerows(rows_to_add)

    after = before + len(rows_to_add)          # we know how many we appended
    print(f"Rows before append: {before}")
    print(f"Appended: {len(rows_to_add)}")
    print(f"Rows after append: {after}")


Rows before append: 224
Appended: 184
Rows after append: 408


In [29]:
# NAV 加上倉別和日期
pod_mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-128GB-PCIe34-TLC5WT-T': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
}

NAV_mappings = {
    'GC-Jetson-AGX64GB-Orin-Industrial-Nvidia-JetPack-6.0': 'GC-J-A64GB-O-Industrial-Nvidia',
    'GC-Jetson-AGX64GB-Orin-Nvidia-JetPack-6.0': 'GC-Jetson-AGX64GB-Orin-Nvidia',
    'AccsyBx-Cardholder-10108GC-5080_70_70Ti': 'AccsyBx-Cardholder-10108GC-5080',
    'Extnd-mPCIeHS_GPS-M800_Mod_Cbl-40CM_kits': 'E-mPCIe-GPS-M800_Mod_40CM',
    'Cbl-M12A5F-OT2-Black-Red-Fuse-100CM': 'Cbl-M12A5F-OT2-B-Red-Fuse-100CM',
    'AccsyBx-Cardholder-9160GC-2000EAda': 'AccsyBx-Cardholder-9160GC-2000E',
    'M.280-SSD-4TB-PCIe4-TLCWT5-NH-IK': 'M.280-SSD-4TB-PCIe4-TLCWT5NH-IK',
    'M.242-SSD-128GB-PCIe34-TLC5WT-TD': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-TD': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.280-SSD-256GB-PCIe44-TLC5WT-TD': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-TD': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'Extnd-mPCIeHS-BTWifi-WT-6218_Mod_Cbl-40CM_kits': 'E-mPCIe-BTWifi-WT-6218_Mod_40CM',
    'GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0': 'GC-Jetson-NX16G-Orin-Nvidia',
    'FPnl-3Ant-NRU-170-PPCseries': 'FPnl-3Ant-NRU-170-PPC series',
}


NAV = pd.read_csv("NAV1.csv", usecols=['Remark', 'QB Num', 'Item', 'Qty(+)', 'Ship Date'], encoding='utf-8')
NAV['Item'] = NAV['Item'].replace(NAV_mappings)
NAV.to_csv('NAV1.csv', index=False)

# 讀取 open purchase2.csv 並處理數據
a = pd.read_csv('open purchase2.csv', usecols=['QB Num', "Order Date", "Inventory Site", "P. O. #", "Name", "Item"])
a.drop_duplicates(inplace=True)
a['Item'] = a['Item'].replace(pod_mappings)
a['Qty(-)'] = "0"

fil = set(a['Item'])
NAV = NAV[NAV['Item'].isin(fil)]
a = a.drop(columns=["Item"])
a.drop_duplicates(inplace=True)

print(len(NAV))
print(len(a))

# 合併 NAV 和 open purchase2.csv
Final = pd.merge(left=NAV, right=a, on=["QB Num"], how="left")
columns = ['Order Date', 'Ship Date', 'QB Num', "P. O. #", "Name", 'Qty(-)', 'Qty(+)', 'Item', 'Inventory Site', 'Remark']
Final
# Final.to_csv('Final.csv', index=False, columns=columns)

print(len(Final))

190
161
190


In [28]:
a

,Order Date,QB Num,P. O. #,Name,Inventory Site,Qty(-)
0,2025/07/24,POD-251046,SO-20251050,Neousys Technology Incorp.,Drop Ship,0
1,2025/07/24,POD-251047,SO-20251051,Neousys Technology Incorp.,Drop Ship,0
2,2025/07/24,POD-251048,SO-20251052,Neousys Technology Incorp.,Drop Ship,0
3,2025/09/18,POD-251338,SO-20251316,Neousys Technology Incorp.,WH01S-NTA,0
4,2025/09/08,POD-251279,SO-20251262,Neousys Technology Incorp.,WH01S-NTA,0
...,...,...,...,...,...,...
460,2025/04/21,POD-250575,SO-20250573,Neousys Technology Incorp.,NaN,0
461,2025/08/28,POD-251228,SO-20251228,Neousys Technology Incorp.,NaN,0
462,2025/09/04,POD-251262,SO-20251258,Neousys Technology Incorp.,NaN,0
463,2025/10/01,POD-251400,SO-20251388,Neousys Technology Incorp.,NaN,0
